In [28]:
from pulp import *
import pandas as pd
import numpy as np

In [171]:
W = ['PadFab','FlecFab','MetRod','PVCRod','MetLeg','PVCLeg','Package']
C = ['MetalCut','PVCCut','FabCut','FabSew','KitAssm']
P = ['36x30M','30x24M','30x24PVC','24x18PVC','36x30Flec','36x30Pad','30x24Flec','30x24Pad','24x18Flec','24x18Pad','36x30MPad','36x30MFlec','30x24MPad','30x24MFlec','30x24PVCPad','30x24PVCFlec','24x18PVCPad',
     '24x18PVCFlec']

In [172]:
M = W + P
R = W + C + P

raw = W + C

In [173]:
T = range(1,8)

In [174]:
T

range(1, 8)

In [175]:
## Reading resources data into a 30*30 matrix (from PabFab to 24x18PVCFlec in order)
## its been converted to its sparse form, by putting 0s for not listed resources in the input data. 

In [176]:
df_1 = pd.read_csv('Bill Of Resources.csv', index_col=0)
resources_bill_dict = makeDict([R,R],df_1.to_numpy())

In [177]:
## Reading supply data into a dictionary

In [178]:
df_2 = pd.read_csv('supply_data.csv', index_col=0)
supply_dict = makeDict([R,T],df_2.to_numpy())

In [179]:
supply_dict

{'PadFab': {1: 3000, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0},
 'FlecFab': {1: 5000, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0},
 'MetRod': {1: 6500, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0},
 'PVCRod': {1: 10500, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0},
 'MetLeg': {1: 200, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0},
 'PVCLeg': {1: 400, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0},
 'Package': {1: 400, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0},
 'MetalCut': {1: 180, 2: 180, 3: 180, 4: 180, 5: 150, 6: 0, 7: 0},
 'PVCCut': {1: 240, 2: 240, 3: 240, 4: 240, 5: 240, 6: 0, 7: 0},
 'FabCut': {1: 240, 2: 240, 3: 240, 4: 300, 5: 300, 6: 0, 7: 0},
 'FabSew': {1: 480, 2: 480, 3: 0, 4: 360, 5: 360, 6: 0, 7: 0},
 'KitAssm': {1: 240, 2: 240, 3: 240, 4: 240, 5: 240, 6: 0, 7: 0},
 '36x30M': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0},
 '30x24M': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0},
 '30x24PVC': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0},
 '24x18PVC': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0},
 '36x30Flec': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0,

In [180]:
## Reading Scrapping cost into a 30*7 matrix 
## (Rows: from PabFab to 24x18PVCFlec in order, Columns: column[0] corresponds to week 1)

In [181]:
df_3 = pd.read_csv('scr_cost.csv', index_col=0)
scrcost_dict = makeDict([R,T],df_3.to_numpy())
scrcost_dict

{'PadFab': {1: 0.02, 2: 0.02, 3: 0.02, 4: 0.02, 5: 0.02, 6: 0.02, 7: 0.02},
 'FlecFab': {1: 0.02, 2: 0.02, 3: 0.02, 4: 0.02, 5: 0.02, 6: 0.02, 7: 0.02},
 'MetRod': {1: 0.01, 2: 0.01, 3: 0.01, 4: 0.01, 5: 0.01, 6: 0.01, 7: 0.01},
 'PVCRod': {1: 0.01, 2: 0.01, 3: 0.01, 4: 0.01, 5: 0.01, 6: 0.01, 7: 0.01},
 'MetLeg': {1: 0.01, 2: 0.01, 3: 0.01, 4: 0.01, 5: 0.01, 6: 0.01, 7: 0.01},
 'PVCLeg': {1: 0.03, 2: 0.03, 3: 0.03, 4: 0.03, 5: 0.03, 6: 0.03, 7: 0.03},
 'Package': {1: 0.05, 2: 0.05, 3: 0.05, 4: 0.05, 5: 0.05, 6: 0.05, 7: 0.05},
 'MetalCut': {1: 0.7, 2: 0.7, 3: 0.7, 4: 0.7, 5: 0.7, 6: 0.7, 7: 0.7},
 'PVCCut': {1: 0.5, 2: 0.5, 3: 0.5, 4: 0.5, 5: 0.5, 6: 0.5, 7: 0.5},
 'FabCut': {1: 0.5, 2: 0.5, 3: 0.5, 4: 0.5, 5: 0.5, 6: 0.5, 7: 0.5},
 'FabSew': {1: 0.5, 2: 0.5, 3: 0.5, 4: 0.5, 5: 0.5, 6: 0.5, 7: 0.5},
 'KitAssm': {1: 0.5, 2: 0.5, 3: 0.5, 4: 0.5, 5: 0.5, 6: 0.5, 7: 0.5},
 '36x30M': {1: 10.0, 2: 10.0, 3: 10.0, 4: 10.0, 5: 10.0, 6: 10.0, 7: 10.0},
 '30x24M': {1: 10.0, 2: 10.0, 3: 10.0, 4: 

In [182]:
## Reading inventory cost into a dictionary
## very high inventory cost (10000) has been modelled for the non inventorible resources. 

In [183]:
df_4 = pd.read_csv('inventory_cost.csv', index_col=0)
inv_dict = makeDict([R,T],df_4.to_numpy())
inv_dict

{'PadFab': {1: 0.005,
  2: 0.005,
  3: 0.005,
  4: 0.005,
  5: 0.005,
  6: 0.005,
  7: 0.005},
 'FlecFab': {1: 0.005,
  2: 0.005,
  3: 0.005,
  4: 0.005,
  5: 0.005,
  6: 0.005,
  7: 0.005},
 'MetRod': {1: 0.006999999999999999,
  2: 0.006999999999999999,
  3: 0.006999999999999999,
  4: 0.006999999999999999,
  5: 0.006999999999999999,
  6: 0.006999999999999999,
  7: 0.006999999999999999},
 'PVCRod': {1: 0.006999999999999999,
  2: 0.006999999999999999,
  3: 0.006999999999999999,
  4: 0.006999999999999999,
  5: 0.006999999999999999,
  6: 0.006999999999999999,
  7: 0.006999999999999999},
 'MetLeg': {1: 0.015,
  2: 0.015,
  3: 0.015,
  4: 0.015,
  5: 0.015,
  6: 0.015,
  7: 0.015},
 'PVCLeg': {1: 0.015,
  2: 0.015,
  3: 0.015,
  4: 0.015,
  5: 0.015,
  6: 0.015,
  7: 0.015},
 'Package': {1: 0.003,
  2: 0.003,
  3: 0.003,
  4: 0.003,
  5: 0.003,
  6: 0.003,
  7: 0.003},
 'MetalCut': {1: 10000.0,
  2: 10000.0,
  3: 10000.0,
  4: 10000.0,
  5: 10000.0,
  6: 10000.0,
  7: 10000.0},
 'PVCCut': {

In [184]:
## Reading demand into a dictionary
## sparse metrix has been modelled for demand as 0 for items not mentioned in the data. 

In [185]:
df_5 = pd.read_csv('demand.csv', index_col=0)
demand_dict = makeDict([R,T],df_5.to_numpy())
demand_dict

{'PadFab': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0},
 'FlecFab': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0},
 'MetRod': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0},
 'PVCRod': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0},
 'MetLeg': {1: 4, 2: 0, 3: 8, 4: 6, 5: 0, 6: 0, 7: 0},
 'PVCLeg': {1: 2, 2: 0, 3: 0, 4: 6, 5: 0, 6: 0, 7: 4},
 'Package': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0},
 'MetalCut': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0},
 'PVCCut': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0},
 'FabCut': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0},
 'FabSew': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0},
 'KitAssm': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0},
 '36x30M': {1: 0, 2: 0, 3: 2, 4: 0, 5: 0, 6: 1, 7: 0},
 '30x24M': {1: 0, 2: 2, 3: 0, 4: 1, 5: 0, 6: 0, 7: 1},
 '30x24PVC': {1: 1, 2: 3, 3: 4, 4: 2, 5: 1, 6: 6, 7: 8},
 '24x18PVC': {1: 0, 2: 2, 3: 0, 4: 3, 5: 0, 6: 4, 7: 6},
 '36x30Flec': {1: 0, 2: 0, 3: 1, 4: 1, 5: 2, 6: 0, 7: 0},
 '36x30Pad': {1: 0, 2: 1, 3: 0, 4: 1, 5: 0, 6: 0, 7: 

In [186]:
## Reading revenue into a dictionary
## sparse metrix has been modelled for revenue as 0 for items not mentioned in the data. 

In [187]:
df_6 = pd.read_csv('revenue.csv', index_col=0)
revenue_dict = makeDict([R,T],df_6.to_numpy())
revenue_dict

{'PadFab': {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0},
 'FlecFab': {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0},
 'MetRod': {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0},
 'PVCRod': {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0},
 'MetLeg': {1: 2.96, 2: 2.96, 3: 2.96, 4: 2.96, 5: 2.96, 6: 2.96, 7: 2.96},
 'PVCLeg': {1: 1.87, 2: 1.87, 3: 1.87, 4: 1.87, 5: 1.87, 6: 1.87, 7: 1.87},
 'Package': {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0},
 'MetalCut': {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0},
 'PVCCut': {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0},
 'FabCut': {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0},
 'FabSew': {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0},
 'KitAssm': {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0},
 '36x30M': {1: 12.49,
  2: 12.49,
  3: 12.49,
  4: 12.49,
  5: 12.49,
  6: 12.49,
  7: 12.49},
 '30x24M': {1: 10.49,
  2: 10.49,
  3: 10.49,
  4: 10.49,


In [188]:
penalty = 0.95
revenue_3D = np.zeros((30, 7, 7))
revenue = df_6.to_numpy()
for i in range(0,30):
    for j in range (0,7):
        for k in range (0,7):
            if(k<j):
                revenue_3D[i][j][k] = 0 
            elif (k==j):
                revenue_3D[i][j][k] = revenue[i][j]
            elif (k>j):
                revenue_3D[i][j][k] = revenue[i][j]*pow(penalty, k-j)

In [189]:
revenue_3D_dict = makeDict([R,T,T],revenue_3D)
revenue_3D_dict

{'PadFab': {1: {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0},
  2: {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0},
  3: {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0},
  4: {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0},
  5: {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0},
  6: {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0},
  7: {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0}},
 'FlecFab': {1: {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0},
  2: {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0},
  3: {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0},
  4: {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0},
  5: {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0},
  6: {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0},
  7: {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0}},
 'MetRod': {1: {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0},
  2: {1: 0.0, 2: 0.0, 

In [190]:
## setting up LP Problem 

In [191]:
prob = LpProblem("Multi period production scheduling", LpMaximize)

In [192]:
## defining LP Variables

In [193]:
sold = LpVariable.dicts("Product_sold", [(product,t,tau) for product in R for t in T for tau in T], lowBound=0, cat='Integer')
produced = LpVariable.dicts("Product_produced", [(product,t) for product in R for t in T], lowBound=0, cat='Integer')
scapped = LpVariable.dicts("Items_scrapped", [(product,t) for product in R for t in T], lowBound=0, cat='Integer')
inventory = LpVariable.dicts("Inventory", [(product,t) for product in R for t in T], lowBound=0, cat='Integer')

In [194]:
## setting up objective function 

In [197]:
prob += lpSum([sold[(product,t,tau)]*revenue_3D_dict[(product,t,tau)] for product in R for t in T for tau in T])
prob -= lpSum([scapped[(product,t)]*scrcost_dict[(product,t)] for product in R for t in T])
prob -= lpSum([inventory[(product,t)]*inv_dict[(product,t)] for product in R for t in T])

KeyError: ('PadFab', 1, 1)

In [198]:
## setting up constraints

In [ ]:
for product in M:
    for t in T:
        prob += lpSum([produced[(product1,t)]*resources_bill_dict[(product1,product2)] for product1 in M for product2 in M for t in T])
            + scapped[(product,t) + inventory[(product,t) 
            + lpSum[sold[(product,t,tau)] for product in R for t in T for tau in range(0,t)]
            == supply_dict[(product,t)] + produced[(product1,t)] + inventory[(product,t-1)

In [ ]:
for product in C:
    for t in T:
        prob += lpSum([produced[(product1,t)]*resources_bill_dict[(product1,product2)] for product1 in M for product2 in M for t in T])
                + scapped[(product,t) == supply_dict[(product,t)]
    

In [ ]:
for product in W:
    for t in T:
        prob += lpSum([produced[(product1,t)]*resources_bill_dict[(product1,product2)] for product1 in M for product2 in M])
            + scapped[(product,t) + inventory[(product,t) 
            + lpSum[sold[(product,t,tau)] for product in R for t in T for tau in range(0,t)]
            == supply_dict[(product,t)] + inventory[(product,t-1)

In [ ]:
for product in M:
    for t in T:

        prob += lpSum([sold[(product,t,tau)]*revenue_3D_dict[(product,t,tau)] for product in R for t in T for tau in T]) 
                <=lpSum[demand[(product,t,tau)] for tau in range(0,t)]

In [ ]:
prob.writeLP("Multi period production scheduling.lp")

In [ ]:
prob.solve()

In [100]:
# for i in M:
#    for t in T:
#        print(supply_dict[i][t])
       